In [1]:
import pandas as pd
import numpy as np
import datetime
import math
import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm

In [2]:
def fix_uniqual_home(games):
    count = 0
    for i in games.iterrows():
        if (i[1]["Unequal Home"].split(',')[0].replace("\'","").replace("[", "") != i[1]["Home"]):
            inter = games.at[i[0],"Unequal Home"]
            games.at[i[0],"Unequal Home"] = games.at[i[0],"Unequal Away"]
            games.at[i[0],"Unequal Away"] = inter
    for i in games.iterrows():
        if (i[1]["Unequal Home"].split(',')[0].replace("\'","").replace("[", "") != i[1]["Home"]):
            print(i)

In [3]:
def fix_all_goals_home(games):
    count = 0
    for i in games.iterrows():
        if (i[1]["All goals Home"].split(',')[0].replace("\'","").replace("[", "") != i[1]["Home"]):
            inter = games.at[i[0],"All goals Home"]
            games.at[i[0],"All goals Home"] = games.at[i[0],"All goals Away"]
            games.at[i[0],"All goals Away"] = inter
    for i in games.iterrows():
        if (i[1]["All goals Home"].split(',')[0].replace("\'","").replace("[", "") != i[1]["Home"]):
            print(i)

In [4]:
def fix_all_missed_goals(games):
    count = 0
    for i in games.iterrows():
        if (i[1]["All missedHome"].split(',')[0].replace("\'","").replace("[", "") != i[1]["Home"]):
            inter = games.at[i[0],"All missedHome"]
            games.at[i[0],"All missedHome"] = games.at[i[0],"All Missed Away"]
            games.at[i[0],"All Missed Away"] = inter
    for i in games.iterrows():
        if (i[1]["All missedHome"].split(',')[0].replace("\'","").replace("[", "") != i[1]["Home"]):
            print(i)

In [5]:
def obtain_data(year):
    games = pd.read_excel('games' + str(year) + ".xlsx")
    fix_uniqual_home(games)
    fix_all_goals_home(games)
    fix_all_missed_goals(games)
    return games

## Game fucntions

In [6]:
def get_linesmen(game):
    linesmen = game['Linesmen'].replace('[','').replace(']', '').split('\',')
    for i in range(len(linesmen)):
        linesmen[i] = linesmen[i].replace('\'', '')
    return linesmen

def get_referes(game):
    referees = game['Referees'].replace('[','').replace(']', '').split('\',')
    for i in range(len(referees)):
        referees[i] = referees[i].replace('\'', '')
    return referees

def get_all_goals(game):
    result = []
    all_goals = game['Goals'].split('],')
    for i in range(len(all_goals)):
        all_goals[i] = all_goals[i].replace('[[', '').replace(' [', '').replace(']]', '')
    for i in range(len(all_goals)):
        time = all_goals[i].split(',')[0].replace('\'', '').replace('′′','').replace('′', ":")
        score = all_goals[i].split(',')[1].replace('\'','').strip()
        goal_equal = all_goals[i].split(',')[2].strip().replace('\'','')
        goal_equal =  0 if goal_equal=='рав.' else 1 if goal_equal=='бол.' else -2 if goal_equal=='бул.' else -1 
        goal = all_goals[i].split(',')[3].strip().replace('\'', '').split(' ')[0]
        assists = []
        if all_goals[i].split(',')[3].strip().replace('\'', '').split(' ')[1]:
            assists.append(all_goals[i].split(',')[3].strip().replace('\'', '').split(' ')[1])
        if all_goals[i].split(',')[3].strip().replace('\'', '').split(' ')[2]:
            assists.append(all_goals[i].split(',')[3].strip().replace('\'', '').split(' ')[2])
        home_currently_on_ice =  all_goals[i].split(',')[4].strip().replace('\'', '').split(' ')
        away_currently_on_ice = all_goals[i].split(',')[5].strip().replace('\'', '').split(' ')
        if home_currently_on_ice[0] == '[]':
            home_currently_on_ice = []
        if away_currently_on_ice[0] == '[]':
            away_currently_on_ice = []
        result.append({"Time":time, "Score": score, "Equality": goal_equal, "Player": goal, "Assists": assists, "Home On Ice": home_currently_on_ice, "Away On Ice": away_currently_on_ice})
    return pd.DataFrame(result)


def get_penalties(penalty_string):
    all_penalties = penalty_string.split('], ')
    result = []
    for i in range(len(all_penalties)):
        all_penalties[i] = all_penalties[i].replace('[[', '').replace('[', '').replace(']]', '')
    for i in range(len(all_penalties)):
        parts = all_penalties[i].split('\',')
        for i in range(len(parts)):
            parts[i] = parts[i].replace('\'', '').strip()
        time = parts[0]
        player = parts[1]
        minutes = parts[2]
        reason = parts[3]
        result.append({"Time": time, "Player": player, "Minutes": minutes, "Reason": reason})
    return pd.DataFrame(result)

def goalkeepers_stats(player_stat):
    parts = player_stat.split('],')
    result = []
    for i in range(len(parts)):
        parts[i] = parts[i].replace('[[[','').replace('[[','').replace('[', '').replace(']', '').replace(']]', '').replace(']]]', '').strip()
    for i in range(2):
        chars = parts[i].split(',')
        for i in range(len(chars)):
            chars[i] = chars[i].replace('\'', '').strip()
        number = chars[0]
        name = chars[1]
        games_count = chars[2]
        wins = chars[3]
        looses = chars[4]
        games_with_shootouts = chars[5]
        shots = chars[6]
        missed_goals = chars[7]
        saves = chars[8]
        percent_saves = chars[9]
        safety_coefficient = chars[10]
        goals = chars[11]
        assists = chars[12]
        no_goals_missed_games = chars[13]
        penalties = chars[14]
        time = chars[15]
        result.append({"Number": number, "Name": name, "Games":games_count,"Wins":wins, "Looses":looses, "Shootout Game": games_with_shootouts,
                      "Shots": shots, "Missed goals": missed_goals, "Saves": saves, "Percent Saves":percent_saves, 
                      "Safety Coefficient": safety_coefficient, "Goals": goals, "Assists": assists, "No Missed Goals": no_goals_missed_games,
                      "Penalties": penalties, "Time": time})
    return pd.DataFrame(result)

def players_stats (player_stat):
    player_stat = player_stat.replace('\', \' "','\'').replace('\"','\'')
    parts = player_stat.split('],')
    results = []
    for i in range(len(parts)):
        parts[i] = parts[i].replace('[[[','').replace('[[','').replace('[', '').replace(']', '').replace(']]', '').replace(']]]', '').strip()
    for i in range(2, len(parts)):
        chars = parts[i].split(',')
        for i in range(len(chars)):
            chars[i] = chars[i].replace('\'', '').replace('\' \' ', '').strip()
        number = chars[0]
        name = chars[1]
        games = chars[2]
        goals_count = chars[3]
        assists = chars[4]
        points = chars[5]
        plus_minus = chars[6]
        plus = chars[7]
        minus = chars[8]
        penalty = chars[9]
        equal_goals = chars[10]
        powerplay_goals = chars[11]
        shorthanded_goals = chars[12]
        OT_goals = chars[13]
        winning_goals = chars[14]
        shootouts_decisive = chars[15]
        shots = chars[16]
        percent_shots = chars[17]
        faceoffs = chars[18]
        win_faceoffs = chars[19]
        percent_faceoff = chars[20]
        time = chars[21]
        shifts = chars[22]
        hits = chars[23]
        block_shots= chars[24]
        foul_against = chars[25]
        time_in_attack = chars[26]
        results.append({"Number": number, "Name":name, "Games":games, "Goals":goals_count, "Assists": assists, "Points": points,
                       "+/-": plus_minus, "+": plus, "-": minus, "Penalties": penalty, "Goals Equal": equal_goals, "PP goal": powerplay_goals,
                       "SO goals": shorthanded_goals, "OT goals": OT_goals, "Win Goals":winning_goals, "Shootout Win": shootouts_decisive,
                       "Shots": shots, "Percent Shots": percent_shots, "Faceoffs":faceoffs, "Win Faceoffs": win_faceoffs,
                       "Percent Faceoffs": percent_faceoff, "Time": time, "Shifts": shifts, "Hits": hits, "Block shots": block_shots,
                       "Fouls Against": foul_against, "time in attack":time_in_attack})
    return pd.DataFrame(results)


def get_uniqual(string):
    parts = string.replace('[','').replace(']','').split(',')
    results = []
    for i in range(len(parts)):
        parts[i] = parts[i].replace('\'','').strip()
    team = parts[0]
    games = parts[1]
    powerplays = parts[2]
    powerplay_goals = parts[3]
    powerplay_percent = parts[4]
    powerplay_missed = parts[5]
    shorthanded = parts[6]
    shorthanded_missed = parts[7]
    shorthanded_percent = parts[8]
    shorthanded_goals = parts[9]
    results.append({"Team": team, "Games": games, "Powerplays": powerplays, "Powerplay Goals": powerplay_goals, 
                   "Powerplays Percent": powerplay_percent, "Poerplay Missed": powerplay_missed, 
                   "Shorthanded": shorthanded, "Shorthanded Missed": shorthanded_missed, "Shorthanded percent": shorthanded_percent,
                   "Shorthanded goals": shorthanded_goals})
    return pd.DataFrame(results)


def get_all_goals_stat(string):
    parts = string.replace('[','').replace(']','').split(',')
    results = []
    for i in range(len(parts)):
        parts[i] = parts[i].replace('\'','').strip()
    team = parts[0]
    games_count = parts[1]
    pl_5_5 = parts[2]
    pl_5_4 = parts[3]
    pl_5_3 = parts[4]
    pl_4_4 = parts[5]
    pl_4_3 = parts[6]
    pl_3_3 = parts[7]
    pl_3_4 = parts[8]
    pl_3_5 = parts[9]
    pl_4_5 = parts[10]
    empty_net = parts[11]
    shootouts = parts[12]
    total = parts[13]
    results.append({"Team": team, "Games": games_count, "5 on 5": pl_5_5, "5 on 4": pl_5_4, "5 on 3": pl_5_3, "4 on 4": pl_4_4,
                  "4 on 3": pl_4_3, "3 on 3": pl_3_3, "3 on 4": pl_3_4, "3 on 5": pl_3_5, "4 on 5": pl_4_5, "Empty net": empty_net,
                  "Shootouts": shootouts, "Total": total})
    return pd.DataFrame(results)

def get_shots (string):
    parts = string.replace('[','').replace(']','').split(',')
    results = []
    for i in range(len(parts)):
        parts[i] = parts[i].replace('\'','').strip()
    periods = []
    current = []
    for i in range(len(parts)):
        if i % 3 == 2:
            current.append(parts[i])
            periods.append(current)
            current = []
        else:
            current.append(parts[i])
    pers = []
    for i in periods:
            pers.append({"Goals": i[0], "Shots":i[1], "Percents":i[2]})
    return pd.DataFrame(pers)  


def players_list(game, is_home):
    stat = players_stats(game['Home Players Stats']) if is_home else players_stats(game['Away Players Stats'])
    return stat[['Name', 'Number']]


def goalie_list(game, is_home):

    stat = goalkeepers_stats(game['Home Players Stats']) if is_home else goalkeepers_stats(game['Away Players Stats'])
    return stat['Name']

def shootouts(index):
    if games.loc[index]['Shootouts']:
        parts = games.loc[index]['Shootouts'].replace('[','').replace(']','').split(',')
        results = []
        current = []
        for i in range(len(parts)):
            parts[i] = parts[i].replace('\'','').strip()
        for i in range(len(parts)):
            if i % 5 == 4:
                current.append(parts[i])
                results.append(current)
                current = []
            else:
                current.append(parts[i])
        goalies_home = goalie_list(index,True)
        goalies_away = goalie_list(index,False)
        players_home = players_list(index,True)
        players_away = players_list(index,False)
        home_SO = []
        away_SO = []
        home_current = 0
        away_current = 0
        for i in results:
            success_home = False
            success_away = False
            goalie_home=""
            goalie_away=""
            player_home=""
            player_away=""
            scores = i[0].split(':')
            if (int(scores[0]) > home_current):
                home_current = home_current + 1
                success_home = True
            if (int(scores[1]) > away_current):
                away_current = away_current + 1
                success_away = True

            name1 = i[1].split('.')[1].strip()
            if (name1 in np.array(goalies_home)):
                goalie_home = name1
            elif (name1 in np.array(goalies_away)):
                goalie_away = name1
            elif(name1 in np.array(players_home)):
                player_home = name1
            elif(name1 in np.array(players_away)):
                player_away = name1
            name2 = i[2].split('.')[1].strip()
            if (name2 in np.array(goalies_home)):
                goalie_home = name2
            elif (name2 in np.array(goalies_away)):
                goalie_away = name2
            elif(name2 in np.array(players_home)):
                player_home = name2
            elif(name2 in np.array(players_away)):
                player_away = name2
            name3 = i[3].split('.')[1].strip()
            if (name3 in np.array(goalies_home)):
                goalie_home = name3
            elif (name3 in np.array(goalies_away)):
                goalie_away = name3
            elif(name3 in np.array(players_home)):
                player_home = name3
            elif(name3 in np.array(players_away)):
                player_away = name3
            name4 = i[4].split('.')[1].strip()
            if (name4 in np.array(goalies_home)):
                goalie_home = name4
            elif (name4 in np.array(goalies_away)):
                goalie_away = name4
            elif(name4 in np.array(players_home)):
                player_home = name4
            elif(name4 in np.array(players_away)):
                player_away = name4
            home_SO.append({"Result":success_home, "Player": player_home, "Goalie": goalie_away})
            away_SO.append({"Result":success_away, "Player": player_away, "Goalie": goalie_home})
        return pd.DataFrame(home_SO), pd.DataFrame(away_SO)
    else:
        return null
    
def get_home_name(index):
    games.loc[index]
    string = games.loc[index]['Unequal Home']
    return get_uniqual(string).loc[0]['Team']
        
    
def get_away_name(index):
    games.loc[index]
    string = games.loc[index]['Unequal Away']
    return get_uniqual(string).loc[0]['Team']


In [7]:
def get_scores(games):
    goals_home = []
    goals_away = []
    for i in range(games.shape[0]):
        goals_home.append(int(np.array(get_all_goals(games.loc[i])['Score'])[-1].split(':')[0]))
        goals_away.append(int(np.array(get_all_goals(games.loc[i])['Score'])[-1].split(':')[1]))
    games['Home Score'] = goals_home
    games['Away Score'] = goals_away

## Seasons Features

In [8]:
difference = 5
seasons = []
for i in range(2009, 2021):
    games = obtain_data(i)
    #get_scores(games)
    seasons.append(games)

#### Goals of home and away teams

In [9]:
for j in range(len(seasons)):
    goals_home = []
    goals_away = []
    for i in seasons[j].iterrows():
        goals_home.append(int(np.array(get_all_goals(i[1])['Score'])[-1].split(':')[0]))
        goals_away.append(int(np.array(get_all_goals(i[1])['Score'])[-1].split(':')[1]))
    seasons[j]["Home Score"] = goals_home
    seasons[j]["Away Score"] = goals_away

#### Overtimes

In [10]:
for j in range(len(seasons)):
    OTs = []
    for i in seasons[j].iterrows():
        if (np.abs(i[1]['Home Score'] -  i[1]['Away Score']) == 1):
            if (int(np.array(get_all_goals(i[1])['Time'])[-1].split(':')[0]) >= 60):
                OTs.append(1)
            else:
                OTs.append(0)
        else:
            OTs.append(0)
    seasons[j]["OT"] = OTs

#### Points Last Games and Season

In [11]:
shift = 5
for j in range(len(seasons)):
    team_points = {}
    points_last_games_home = []
    points_last_games_away = []
    points_home = []
    points_away = []
    for i in seasons[j]["Home"].unique():
        team_points.update({i: [0]})

    
    for i in seasons[j].iterrows():
        if len(team_points[i[1]["Away"]]) - 1 - shift >= 0:
            points_last_games_away.append(team_points[i[1]["Away"]][-1] -  team_points[i[1]["Away"]][-1 - shift])
        else:
            points_last_games_away.append(0)

        if len(team_points[i[1]["Home"]]) - 1 - shift >= 0:
            points_last_games_home.append(team_points[i[1]["Home"]][-1] -  team_points[i[1]["Home"]][-1 - shift])
        else:
            points_last_games_home.append(0)
        
        points_home.append(team_points[i[1]["Home"]][-1])
        points_away.append(team_points[i[1]["Away"]][-1])
        
        if ((i[1]["Home Score"] < i[1]["Away Score"]) & (i[1]["OT"] == 0)):
            team_points[i[1]["Away"]].append(team_points[i[1]["Away"]][-1] + 3)
            team_points[i[1]["Home"]].append(team_points[i[1]["Home"]][-1])
               
        if ((i[1]["Home Score"] < i[1]["Away Score"]) & (i[1]["OT"] == 1)):
            team_points[i[1]["Home"]].append(team_points[i[1]["Home"]][-1] + 1)
            team_points[i[1]["Away"]].append(team_points[i[1]["Away"]][-1] + 2)

        if ((i[1]["Home Score"] > i[1]["Away Score"]) & (i[1]["OT"] == 1)):
            team_points[i[1]["Home"]].append(team_points[i[1]["Home"]][-1] + 2)
            team_points[i[1]["Away"]].append(team_points[i[1]["Away"]][-1] + 1)        


        if ((i[1]["Home Score"] > i[1]["Away Score"]) & (i[1]["OT"] == 0)):
            team_points[i[1]["Home"]].append(team_points[i[1]["Home"]][-1] + 3)
            team_points[i[1]["Away"]].append(team_points[i[1]["Away"]][-1])
    
    
    seasons[j]["Points Last Games Home"] = points_last_games_home
    seasons[j]["Points Last Games Away"] = points_last_games_away
    seasons[j]["Home Points"] = points_home
    seasons[j]["Away Points"] = points_away

#### Home/Away team points 

Calculate percent of points of home/away team, that were gotten in their home/away games

In [12]:
for j in range(len(seasons)):
    teams_home = {}
    teams_away = {}
    teams_home_games = {}
    teams_away_games = {}
    for i in seasons[j]["Home"].unique():
        teams_home.update({i: 0})
        teams_away.update({i :0})
        teams_home_games.update({i: 0})
        teams_away_games.update({i: 0})
    points_percent_home = []
    points_percent_away = []
    for i in seasons[j].iterrows():
        try:
            points_percent_home.append(teams_home[i[1]["Home"]]/(3*teams_home_games[i[1]["Home"]]))
        except:
            points_percent_home.append(0)
        try:
            points_percent_away.append(teams_away[i[1]["Away"]]/(3*teams_home_games[i[1]["Away"]]))
        except:
            points_percent_away.append(0)
        teams_home_games[i[1]["Home"]] = teams_home_games[i[1]["Home"]] + 1
        teams_away_games[i[1]["Away"]] = teams_away_games[i[1]["Away"]] + 1
        if ((i[1]["Home Score"] < i[1]["Away Score"]) & (i[1]["OT"] == 0)):
            teams_away[i[1]["Away"]] = teams_away[i[1]["Away"]] + 3
            
            
            
        if ((i[1]["Home Score"] < i[1]["Away Score"]) & (i[1]["OT"] == 1)):
            teams_home[i[1]["Home"]] = teams_home[i[1]["Home"]] + 1
            teams_away[i[1]["Away"]] = teams_away[i[1]["Away"]] + 2

        if ((i[1]["Home Score"] > i[1]["Away Score"]) & (i[1]["OT"] == 1)):
            teams_home[i[1]["Home"]] = teams_home[i[1]["Home"]] + 2
            teams_away[i[1]["Away"]] = teams_away[i[1]["Away"]] + 1        


        if ((i[1]["Home Score"] > i[1]["Away Score"]) & (i[1]["OT"] == 0)):
            teams_home[i[1]["Home"]] = teams_home[i[1]["Home"]] + 3

        
    
    
    seasons[j]["Points Percent Home"] = points_percent_home
    seasons[j]["Points Percent Away"] = points_percent_away

#### Target column

Did home team win in regular? 

In [13]:
for j in range(len(seasons)):
    
    results = []
    for i in seasons[j].iterrows():
        if (i[1]["Home Score"] > i[1]['Away Score']) & (i[1]["OT"] == 0):
            results.append(1)
        elif (i[1]["Home Score"] < i[1]['Away Score']) & (i[1]["OT"] == 0):
            results.append(0)
        elif (i[1]["OT"] == 1):
            results.append(0)
    seasons[j]["Result"] = results
    seasons[j]['Score Ratio'] = seasons[j]['Home Points']/seasons[j]['Away Points']
    seasons[j]['Score Ratio'] = seasons[j]['Score Ratio'].fillna(1)

#### Power Rating

This rating shows how the teams play last time. If one of team wins another, the winner gets ratio_1 percent of points of defeated team and the loser loses the same number of points. If victory was in overtime, number of points is equal to ratio_2 percent of points of defeated team 

In [14]:
ratio_1 = 0.5
ratio_2 = 0.25
for j in range(len(seasons)):
    teams = {}
    for i in seasons[j]["Home"].unique():
        teams.update({i: 200})
    power_rating_home = []
    power_rating_away = []

    for i in seasons[j].iterrows():
        power_rating_home.append(teams[i[1]["Home"]])
        power_rating_away.append(teams[i[1]["Away"]])
        if ((i[1]["Home Score"] < i[1]["Away Score"]) & (i[1]["OT"] == 0)):
            transfer = teams[i[1]["Home"]] * ratio_1
            teams[i[1]["Away"]] = teams[i[1]["Away"]] + transfer
            teams[i[1]["Home"]] = teams[i[1]["Home"]] - transfer
        if ((i[1]["Home Score"] < i[1]["Away Score"]) & (i[1]["OT"] == 1)):
            transfer = teams[i[1]["Home"]] * ratio_2
            teams[i[1]["Away"]] = teams[i[1]["Away"]] + transfer
            teams[i[1]["Home"]] = teams[i[1]["Home"]] - transfer
        if ((i[1]["Home Score"] > i[1]["Away Score"]) & (i[1]["OT"] == 1)):
            transfer = teams[i[1]["Away"]] * ratio_2
            teams[i[1]["Away"]] = teams[i[1]["Away"]] - transfer
            teams[i[1]["Home"]] = teams[i[1]["Home"]] + transfer
        if ((i[1]["Home Score"] > i[1]["Away Score"]) & (i[1]["OT"] == 0)):
            transfer = teams[i[1]["Away"]] * ratio_1
            teams[i[1]["Away"]] = teams[i[1]["Away"]] - transfer
            teams[i[1]["Home"]] = teams[i[1]["Home"]] + transfer

    seasons[j]["Home Power Rating"] = power_rating_home
    seasons[j]["Away Power Rating"] = power_rating_away


## Player Features

In [15]:
goals = get_all_goals(seasons[0].loc[6])

In [16]:
# for s in range(len(seasons)):
    
#     teams_goals = {}
#     teams_assists = {}
#     teams_positive = {}
#     teams_negative = {}
#     goals_home_optimal = []
#     goals_away_optimal = []
#     assists_home_optimal = []
#     assists_away_optimal = []
    
    
#     for i in seasons[s]["Home"].unique():
#         teams_goals.update({i: {}})
#         teams_assists.update({i: {}})
#         teams_positive.update({i: {}})
#         teams_negative.update({i: {}})
#         for j in range(1,100,1):
#             teams_goals[i].update({str(j): 0}) 
#             teams_assists[i].update({str(j): 0}) 
#             teams_positive[i].update({str(j):0})
#             teams_negative[i].update({str(j):0})
            

#     for i in seasons[s].iterrows():
#         players_home = players_list(i[1], True)
#         players_away = players_list(i[1], False)


#         goals = get_all_goals(i[1])
#         home = i[1]["Home"]
#         away = i[1]["Away"]
#         current_score = [0,0]

#         goals_roster_home = 0
#         goals_roster_home_total = 0
#         goals_roster_away = 0
#         goals_roster_away_total = 0
        
        
#         assists_roster_away = 0
#         assists_roster_away_total = 0
#         assists_roster_home = 0
#         assists_roster_home_total = 0




#         for j in teams_goals[home].keys():
#             goals_roster_home_total = goals_roster_home_total + teams_goals[home][j]
            
#         for j in teams_goals[away].keys():
#             goals_roster_away_total = goals_roster_away_total + teams_goals[away][j]
            
#         for j in teams_assists[home].keys():
#             assists_roster_home_total = assists_roster_home_total + teams_assists[home][j]
            
#         for j in teams_assists[away].keys():
#             assists_roster_away_total = assists_roster_away_total + teams_assists[away][j]
            
            
            
#         for j in players_home['Number']:
#             goals_roster_home = goals_roster_home + teams_goals[home][j]
#             assists_roster_home = assists_roster_home + teams_assists[home][j]

#         for j in players_away['Number']:
#             goals_roster_away = goals_roster_away + teams_goals[away][j]
#             assists_roster_away = assists_roster_away + teams_assists[away][j]

            
            
            
#         if (goals_roster_home_total!= 0):    
#             goals_home_optimal.append(goals_roster_home/goals_roster_home_total)
#         else:
#             goals_home_optimal.append(0)
#         if (goals_roster_away_total!= 0):
#             goals_away_optimal.append(goals_roster_away/goals_roster_away_total)
#         else:
#             goals_away_optimal.append(0)
            
            
            
#         if (assists_roster_home_total!= 0):    
#             assists_home_optimal.append(assists_roster_home/assists_roster_home_total)
#         else:
#             assists_home_optimal.append(0)
#         if (assists_roster_away_total!= 0):
#             assists_away_optimal.append(assists_roster_away/assists_roster_away_total)
#         else:
#             assists_away_optimal.append(0)
            
            
            
            

#         for j in goals.iterrows():
#             if j[1]['Time'] != '65:00': 
#                 new_current_score = j[1]["Score"].split(":")
#                 new_current_score[0] = int(new_current_score[0]) 
#                 new_current_score[1] = int(new_current_score[1])
#                 goal_home = True
#                 if current_score[0] == new_current_score[0]:
#                     goal_home = False
#                 if goal_home:
#                     teams_goals[home][j[1]["Player"]] = teams_goals[home][j[1]["Player"]] + 1
#                     for assist in j[1]["Assists"]:
#                         teams_assists[home][assist] = teams_assists[home][assist] + 1
#                     for positive in j[1]['Home On Ice']:
#                         teams_positive[home][positive] = teams_positive[home][positive] + 1
#                     for negative in j[1]['Away On Ice']:
#                         teams_negative[away][negative] = teams_negative[away][negative] + 1
#                 else:
#                     teams_goals[away][j[1]["Player"]] = teams_goals[away][j[1]["Player"]] + 1
#                     for assist in j[1]["Assists"]:
#                         teams_assists[away][assist] = teams_assists[away][assist] + 1
#                     for positive in j[1]['Away On Ice']:
#                         teams_positive[away][positive] = teams_positive[away][positive] + 1
#                     for negative in j[1]['Home On Ice']:
#                         teams_negative[home][negative] = teams_negative[home][negative] + 1
#                 current_score = new_current_score


#     seasons[s]["Goals Home"] = goals_home_optimal
#     seasons[s]["Goals Away"] = goals_away_optimal
#     seasons[s]["Assists Home"] = assists_home_optimal
#     seasons[s]["Assists Away"] = assists_away_optimal
    
    
    #if (home == "Авангард"):
    #    print (away + " " + str(teams_goals[home]['20']))
    #if (away == "Авангард"):
    #    print (home + " " + str(teams_goals[away]['20']))


In [17]:
def get_player_statistic_shape(ch_name):
    result_home = []
    result_away = []

    
    for s in tqdm(range(len(seasons))):
        teams_characteristic = {}
        result_characteristic = []
        for i in seasons[s]["Home"].unique():
            teams_characteristic.update({i: {}})
            for j in range(1,100,1):
                teams_characteristic[i].update({str(j): 0}) 
        charasteristic_home_optimal = []
        charasteristic_away_optimal = []
        for i in seasons[s].iterrows():
                players_home = players_list(i[1], True)
                players_away = players_list(i[1], False)
                characteristic_home = players_stats(i[1]["Home Players Stats"])[[ch_name, "Number"]]
                charasteristic_away = players_stats(i[1]["Away Players Stats"])[[ch_name, "Number"]]
                home = i[1]["Home"]
                away = i[1]["Away"]


                charasteristic_roster_home = 0
                for j in characteristic_home.iterrows():
                    charasteristic_roster_home = charasteristic_roster_home + teams_characteristic[home][j[1]["Number"]]

                charasteristic_roster_away = 0
                for j in charasteristic_away.iterrows():
                    charasteristic_roster_away = charasteristic_roster_away + teams_characteristic[away][j[1]["Number"]]

                charasteristic_roster_home_total = 0
                for j in teams_characteristic[home].keys():
                    charasteristic_roster_home_total = charasteristic_roster_home_total + teams_characteristic[home][j]

                charasteristic_roster_away_total = 0
                for j in teams_characteristic[away].keys():
                    charasteristic_roster_away_total = charasteristic_roster_away_total + teams_characteristic[away][j]




                if (charasteristic_roster_home_total!= 0):   
                    charasteristic_home_optimal.append(charasteristic_roster_home/charasteristic_roster_home_total)

                else:
                    charasteristic_home_optimal.append(0)



                if (charasteristic_roster_away_total!= 0):
                    charasteristic_away_optimal.append(charasteristic_roster_away/charasteristic_roster_away_total)
                else:
                    charasteristic_away_optimal.append(0)


                for j in characteristic_home.iterrows():
                    try:
                        teams_characteristic[home][j[1]["Number"]] = teams_characteristic[home][j[1]["Number"]] + int(j[1][ch_name])
                    except:
                        print(i)
                        print(players_stats(i[1]["Home Players Stats"]))
                        print(players_stats(i[1]["Away Players Stats"]))
                        return null
                
                
                
                for j in charasteristic_away.iterrows():
                    try:
                        teams_characteristic[away][j[1]["Number"]] = teams_characteristic[away][j[1]["Number"]] + int(j[1][ch_name])
                    except:
                        print(i)
                        print(players_stats(i[1]["Home Players Stats"]))
                        print(players_stats(i[1]["Away Players Stats"]))
                        return null

        result_home.append(charasteristic_home_optimal)
        result_away.append(charasteristic_away_optimal)
    return result_home,result_away
            
            
            


In [18]:
#"Time": time, "Shifts": shifts, "Hits": hits, "Block shots": block_shots,
#"Fouls Against": foul_against, "time in attack":time_in_attack})

In [19]:
statistic_names = ["Goals", 'Assists', "Points", "+/-", "+", "-", "Penalties", "PP goal", "Goals Equal", 
                  "SO goals", "OT goals", "Win Goals", "Shootout Win", "Shots", "Faceoffs", "Win Faceoffs"]

In [21]:
for name in statistic_names:
    res_home, res_away = get_player_statistic_shape(name)
    for i in range(len(res_home)):
        seasons[i][name + " Home"] = res_home[i]
        seasons[i][name + " Away"] = res_away[i]

100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [02:30<00:00, 12.58s/it]


In [22]:
seasons[0]

,Date,Home,Away,Home Coach,Away Coach,Attendance,Referees,Linesmen,Goals,Shootouts,...,OT goals Home,OT goals Away,Win Goals Home,Win Goals Away,Shootout Win Home,Shootout Win Away,Faceoffs Home,Faceoffs Away,Win Faceoffs Home,Win Faceoffs Away
0,02.09.2008,Салават Юлаев,Локомотив,Михалёв Сергей,Хейккиля Кари,8400,"['Буланов Вячеслав', 'Бирюков Андрей']","['Зибинин Иван', 'Шемякин Алексей']","[['18′33′′', '0:1', 'рав.', '72 4 8', '30 45 2...",[],...,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
1,02.09.2008,Амур,Динамо Р,Емелин Анатолий,Шуплер Юлиус,7100,['Оленин Константин'],"['Медведев Алексей', 'Бирин Виктор']","[['12′15′′', '0:1', 'бол.', '17 23 24', '29 15...",[],...,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
2,02.09.2008,Металлург Нк,Динамо Мн,Михайлов Борис,Хьюз Джим,5900,['Раводин Алексей'],"['Копосов Алексей', 'Небольсин Михаил']","[['05′38′′', '0:1', 'бол.', '68 71 79', '39 12...",[],...,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
3,02.09.2008,Спартак,Атлант,Ржига Милош,Канарейкин Фёдор,3200,"['Кадыров Рафаэль', 'Кулаков Сергей']","['Камуркин Александр', 'Шелянин Сергей']","[['06′29′′', '0:1', 'мен.', '27 29 ', '44 2 14...",[],...,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
4,02.09.2008,Витязь,ХК МВД,Гомоляко Сергей,Знарок Олег,2500,['Захаров Анатолий'],"['Садовников Александр', 'Сазонов Иван']","[['01′06′′', '0:1', 'рав.', '23 51 ', '30 47 5...","['0:0', '47. Носов Антон', '30. Бирюков Михаил...",...,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,26.02.2009,Северсталь,Металлург Мг,Пятанов Андрей,Белоусов Валерий К.,2800,"['Гашилов Виктор', 'Оленин Константин']","['Пьянков Константин', 'Голдырев Максим']","[['24′01′′', '0:1', 'бол.', '15 60 27', '35 4 ...",[],...,1.0,1.0,0.650000,0.884615,0.428571,1.00,0.746713,0.977949,0.729600,0.981886
668,26.02.2009,Локомотив,Трактор,Хейккиля Кари,Назаров Андрей,8800,['Феофанов Сергей'],"['Пахомов Андрей', 'Ягубкин Иван']","[['02′48′′', '1:0', 'бол.', '63 19 22', '20 5 ...",[],...,1.0,0.0,0.909091,0.869565,1.000000,0.50,0.959255,0.755316,0.962282,0.760510
669,26.02.2009,Динамо Р,Амур,Шуплер Юлиус,Емелин Анатолий,6500,['Горский Алексей'],"['Михель Алексей', 'Шиханов Роман']","[['48′41′′', '0:1', 'бол.', '72 91 ', '35 15 1...",[],...,1.0,1.0,0.777778,0.875000,1.000000,1.00,0.776625,0.845134,0.784574,0.853484
670,26.02.2009,Динамо Мн,Металлург Нк,Спиридонов Василий,Михайлов Борис,2700,['Васильев Алексей'],"['Кулёв Артур', 'Томилов Виктор']","[['05′30′′', '1:0', 'бол.', '8 75 86', '1 8 40...",[],...,0.0,1.0,0.769231,0.714286,0.000000,0.50,0.562450,0.821514,0.568451,0.844886


In [25]:
for i in range(len(seasons)):
    seasons[i].to_csv("Season" + str(i)+ ".csv")

In [23]:
seasons[0].columns

Index(['Date', 'Home', 'Away', 'Home Coach', 'Away Coach', 'Attendance',
       'Referees', 'Linesmen', 'Goals', 'Shootouts', 'Penalties Home',
       'Penalties Away', 'Home Players Stats', 'Away Players Stats',
       'Unequal Home', 'Unequal Away', 'All goals Home', 'All goals Away',
       'All missedHome', 'All Missed Away', 'Shots Home', 'Shots Away',
       'Final Status', 'Home Score', 'Away Score', 'OT',
       'Points Last Games Home', 'Points Last Games Away', 'Home Points',
       'Away Points', 'Points Percent Home', 'Points Percent Away', 'Result',
       'Score Ratio', 'Home Power Rating', 'Away Power Rating', 'Goals Home',
       'Goals Away', 'Assists Home', 'Assists Away', 'Points Home',
       'Points Away', '+/- Home', '+/- Away', '+ Home', '+ Away', '- Home',
       '- Away', 'PP goal Home', 'PP goal Away', 'Goals Equal Home',
       'Goals Equal Away', 'SO goals Home', 'SO goals Away', 'OT goals Home',
       'OT goals Away', 'Win Goals Home', 'Win Goals Away',
  